##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Classification using RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/sequences/text_classification_using_rnn"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/sequences/text_classification_using_rnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/sequences/text_classification_using_rnn.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this tutorial, we will do text classification using Recurrent Neural Networks on the imdb dataset.

In [0]:
from __future__ import absolute_import, division, print_function

!pip install tf-nightly-2.0-preview
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

## Downloading the data

The imdb dataset is a binary classification dataset. All the reviews are either positive or negative reviews.

In [0]:
imdb = tf.keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [0]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

index_word = dict([(value, key) for (key, value) in word_index.items()])

In [0]:
def decode_review(text):
    return ' '.join([index_word.get(i, '?') for i in text])

# pad the data to a fixed length
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data,
                                                           value=word_index["<PAD>"],
                                                           padding='post',
                                                           maxlen=256)

test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data,
                                                          value=word_index["<PAD>"],
                                                          padding='post',
                                                          maxlen=256)

In [0]:
len(train_data[0]), len(test_data[0])

In [0]:
decode_review(train_data[0])

In [0]:
# helper function to plot graphs
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])

In [0]:
# creating the training and validation sets
vocab_size = 10000

x_val = train_data[:10000]
x_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

In [0]:
BUFFER_SIZE = len(train_data)
BATCH_SIZE = 64

## Creating an input data pipeline using tf.data

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).batch(BATCH_SIZE)

## Creating the model

*    Let's first create a embedding layer.  An embedding layer organize the words with similar meanings into similar vectors. Comparing to one-hot encoding, word embedding is dense and relative low dimensional making it faster to train. Moreover in an embedding, words with similar meaning are close to each other in the multi-dimensional space.

*    A Recurrent layer process sequence input by iterating through the elements in the sequence, and maintain a state for all the data it has seen so far. Here we are using Long Short Term Memory (LSTM).





In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_dataset, epochs=10, 
                    steps_per_epoch=len(x_train)//BATCH_SIZE, 
                    validation_data=test_dataset,
                    validation_steps=len(x_val)//BATCH_SIZE)

In [0]:
test_loss, test_acc = model.evaluate(test_dataset, steps=len(test_data)//BATCH_SIZE)
print ('Test Loss: {}'.format(test_loss))
print ('Test Accuracy: {}'.format(test_acc))

In [0]:
plot_graphs(history, 'acc')

In [0]:
plot_graphs(history, 'loss')

## Stacking 2 or more LSTM layers

Keras recurrent layers can be run in two different modes: they return either the full sequences of successive outputs for each timestep (a 3D tensor of shape (batch_size, timesteps, output_features)), or return only the last output for each input sequence (a 2D tensor of shape (batch_size, output_features)). These two modes are controlled by the return_sequences constructor argument.

You can also use the Bidirectional wrapper across an LSTM layer. This propagates the input forward and backwards via the RNN layer and then concatenates the output. This helps the RNN to learn long range dependencies.

e.g.: **tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(...))**


In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_dataset, epochs=10, 
                    steps_per_epoch=len(x_train)//BATCH_SIZE, 
                    validation_data=test_dataset,
                    validation_steps=len(x_val)//BATCH_SIZE)

In [0]:
test_loss, test_acc = model.evaluate(test_dataset, steps=len(test_data)//BATCH_SIZE)
print ('Test Loss: {}'.format(test_loss))
print ('Test Accuracy: {}'.format(test_acc))

In [0]:
plot_graphs(history, 'acc')

In [0]:
plot_graphs(history, 'loss')

## Next steps

* Check other existing recurrent layers such as [GRU Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU).
* Try improving the network accuracy, and prevent overfitting.



